# 2.1 Explicit Intent

W tym przykładzie przyjrzymy się mechanizmowi intentów, rozpoczniemy od intentów jawnych. Stworzymy aplikację z dwiema aktywnościami i umożliwimy otworzenie drugiej aktywności z poziomu pierwszej (głównej). 

Przedźmy do aktywności głównej i dodajmy przycisk do layoutu.

```xml
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".MainActivity">

    <Button
        android:id="@+id/button"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:text="Button"
        android:layout_margin="16dp"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintEnd_toEndOf="parent"
        android:onClick="startSecondActivity"/>
</androidx.constraintlayout.widget.ConstraintLayout>
```

Po naciśnięciu tego przycisku możliwe będzie przejście do drugiej aktywności.

Następnie dodajmy drugą aktywność do projektu. W oknie **Project** wybieram z menu kontekstowego **New -> Activity -> Empty Activity**. Zaznaczam **Generate layout file** i odznaczam **Launcher Activity**. Przejdźmy do pliku `AndroidManifest.xml`

Mamy opisane dwie aktywności. Chcemy teraz wskazać rodzica dla **SecondActivity** . Do tagu `<activity />` dodaję opcję `android:parentActivityName=".MainActivity"` oraz `android:label="Second Activity"`.

```xml
<activity
    android:name=".SecondActivity"
    android:parentActivityName=".MainActivity"
    android:label="Second Activity"
    android:exported="false" />
```

Czyli wskazaliśmy ze aktywnością nadrzędną jest **MainActivity**. Zwróćmy również uwagę na kropkę przed nazwą aktywności głównej - pominięta została nazwa pakietu (`pl.edu.uwr.pum.explicitintentjava.MainActivity`). `label` posłuży jako wyświetlana nazwa aktywności.

Teraz utworzymy prosty layout dla drugiej aktywności - przechodzimy do pliku `activity_second.xml` i dodajemy pole `TextView`.

```xml
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".SecondActivity">

    <TextView
        android:id="@+id/textView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:text="Second Activity"
        android:textSize="40sp"
        android:textStyle="bold"
        android:textColor="@color/black"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />
</androidx.constraintlayout.widget.ConstraintLayout>
```

Dodajmy teraz obsługę przycisku w naszej głównej aktywności, przyjdźmy do pliku `MainActivity.java`. Po wciśnięciu przycisku chcemy przejść do drugiej aktywności.

```java
public void startSecondActivity(View view) {
        
}
```

W pierwszym kroku musimy utworzyć **intent**.

```java
Intent intent = new Intent(this, SecondActivity.class);
```

Konstruktor klasy `Intent` przyjmuje dwa argumenty. Pierwszym jest **context**, ponieważ aktywność jest swoim własnym kontekstem używamy słowa kluczowego `this`. Drugim argumentem jest klasa aktywności którą chcemy włączyć - tutaj będzie to `SecondActivity.class`.

Aby włączyć aktywność wykonujemy metodę `startActivity`, jako argumentb podajemy poprzednio utworzony `Intent`.

```java
startActivity(intent);
```

Na tym etapie możemy zbudować nasz projekt i przetestować funkcjonalność przycisku. Po przejściu do drugiej aktywności zwróćmy uwagę na przycisk powrotu znajdujący się na belce po lewej stronie.

<img src="images/explicitIntent1.png" width="200"/>